In [1]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plot
from glob import glob

In [2]:
IMAGESHAPE = [224, 224, 3] 
training_data = 'chest_xray/train'
testing_data = 'chest_xray/test'

In [3]:
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 11s 0us/step


In [4]:
for each_layer in vgg_model.layers:
    each_layer.trainable = False

In [5]:
classes = glob('chest_xray/train/*') 

In [6]:

flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)

In [7]:

final_model = Model(inputs=vgg_model.input, outputs=prediction) 
final_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
final_model.compile( 
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [9]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)

In [10]:
training_set = train_datagen.flow_from_directory('chest_xray/train', 
                                                 target_size = (224, 224),
                                                 batch_size = 4,
                                                 class_mode = 'categorical')

Found 5216 images belonging to 2 classes.


In [11]:

test_set = testing_datagen.flow_from_directory('chest_xray/test',
                                               target_size = (224, 224),
                                               batch_size = 4,
                                               class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [13]:
fitted_model = final_model.fit_generator( 
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\SUNNYK~1\AppData\Local\Temp/ipykernel_17184/952933750.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fitted_model = final_model.fit_generator(


Epoch 1/5
1304/1304 [==============================] - 2413s 2s/step - loss: 0.3233 - accuracy: 0.9331 - val_loss: 0.8032 - val_accuracy: 0.8942
Epoch 2/5
1304/1304 [==============================] - 2874s 2s/step - loss: 0.2539 - accuracy: 0.9484 - val_loss: 0.4702 - val_accuracy: 0.9151
Epoch 3/5
1304/1304 [==============================] - 2800s 2s/step - loss: 0.1953 - accuracy: 0.9576 - val_loss: 0.6253 - val_accuracy: 0.9071
Epoch 4/5
1304/1304 [==============================] - 2796s 2s/step - loss: 0.2566 - accuracy: 0.9544 - val_loss: 0.7556 - val_accuracy: 0.9199
Epoch 5/5
1304/1304 [==============================] - 2159s 2s/step - loss: 0.2052 - accuracy: 0.9618 - val_loss: 1.5079 - val_accuracy: 0.8670


In [14]:
final_model.save('savved_model.h5')

In [16]:
final_model.save("sunny")

INFO:tensorflow:Assets written to: sunny\assets
